# Data Loading
## MSc Advanced Analytics Dissertation

Here we are looking at taking several samples of the amazon reviews dataset and loading them into a dataframe.

In [1]:
# get modules in 
import pandas as pd
import gzip

# Datasets



# Quick look at a Dataset

## 5-K Fashion Dataset 
    (AMAZON_FASHION_5.json)

Format is one-review-per-line in json.

- **overall**: ratings of the product
- **reviewerID**: ID of the reviewer, e.g. A2SUAM1J3GNN3B
- **asin**: ID of the product, e.g. 0000013714
- **reviewerName**: name of the reviewer
- **vote**: helpful votes of the review
- **style**: a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- **reviewText**: text of the review
- **summary**: summary of the review
- **unixReviewTime**: time of the review (unix time)
- **reviewTime**: time of the review (raw)
- **image**: images that users post after they have received the product



In [6]:
# data path
fashion_data = "/Users/pavansingh/Library/CloudStorage/GoogleDrive-sngpav003@myuct.ac.za/My Drive/Masters 2022/Dissertation/Masters-Dissertation/Data/AMAZON_FASHION_5.json"
fashion_data = pd.read_json(fashion_data, lines=True)
fashion_data = fashion_data.loc[:,['reviewerName', 'reviewText', 'overall', 'style']]
display(fashion_data.loc[10:20,:])
print("Shape of Data:", fashion_data.shape)

,reviewerName,reviewText,overall,style
10,Denise A. Conte,Relieved my Plantar Fascitis for 3 Days. Then ...,2,"{'Size Name:': ' Men's 6-6.5, Women's 8-8.5', ..."
11,Cognizant Consumer,This is my 6th pair and they are the best thin...,2,"{'Size Name:': ' Men's 8-8.5, Women's 10-10.5'..."
12,William_Jasper,We have used these inserts for years. They pr...,5,NaN
13,Cognizant Consumer,Pinnacle seems to have more cushioning so my h...,5,NaN
14,William_Jasper,Excellent insole with good support.,5,NaN
15,Cognizant Consumer,A little more cushion than the Powerstep Prote...,5,NaN
16,William_Jasper,These insoles help my heels feel much better. ...,5,NaN
17,Denise A. Conte,Relieved my Plantar Fascitis for 3 Days. Then ...,2,NaN
18,Cognizant Consumer,This is my 6th pair and they are the best thin...,2,NaN
19,William_Jasper,We have used these inserts for years. They pr...,5,NaN


Shape of Data: (3176, 4)


In [10]:
fashion_data['style'].isna().sum()
fashion_data.dropna(subset=["style"], inplace=True)
fashion_data.sort_values(by=['overall'], inplace=True, ascending=False)
display(fashion_data.head(10))
print("Shape of Data:", fashion_data.shape)

,reviewerName,reviewText,overall,style
2820,Kitten,"Absolutly love these shoes!! Beautiful color, ...",5,"{'Size:': ' 8 M US', 'Color:': ' Energy/Mtlc S..."
966,Amazon Customer,Absolutely love these shoes and color!! They a...,5,"{'Size:': ' 8 B(M) US', 'Color:': ' Hyper Jade..."
987,dan h.,Good shoes,5,"{'Size:': ' 7 B(M) US', 'Color:': ' Black/Whit..."
989,Mel B,BEST sneakers I've ever purchased!!!!,5,"{'Size:': ' 8 B(M) US', 'Color:': ' Black/Whit..."
990,Amazon Customer,Excellent ...!!!,5,"{'Size:': ' X-Large', 'Color:': ' Wolf Grey/Pl..."
991,gi,the best sneakers by far! I had never owned a ...,5,"{'Size:': ' 8.5 B(M) US', 'Color:': ' Black/Wh..."
992,Bree,Love these shoes. I've been buying Nikes for y...,5,"{'Size:': ' 9 B(M) US', 'Color:': ' Black/Whit..."
995,Flip Flop Life,So lightweight and awesome!,5,"{'Size:': ' 11 B(M) US', 'Color:': ' Black/Whi..."
973,Amazon Customer,"On time, nice sneakers. Fit as expected.",5,"{'Size:': ' 7.5 B(M) US', 'Color:': ' Black/Gr..."
972,judy laurel,"Cute shoe, fit as expected, and very comfortable.",5,"{'Size:': ' 8.5 B(M) US', 'Color:': ' Black/Wh..."


Shape of Data: (3107, 4)


# All Category Data

Using DataFiniti's Product Database

In [4]:
# load data sets from csv
data = "/Users/pavansingh/Desktop/Amazon Review Data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"
data = pd.read_csv(data)
#data = data.loc[:,['reviewerName', 'reviewText', 'overall', 'style']]
#display(data.loc[10:20,:])
print("Shape of Data:", data.shape)

Shape of Data: (28332, 24)


'I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.'